

# AMLS Assignment
## Task B_2: CNN Hyper on BloodMNIST Dataset

Explore CNN hyperparameter set selection for the BloodMNIST dataset.

## Import libraries
The required libraries for this notebook are sklearn, tensorflow, numpy and matplotlib.

In [1]:
## first enable autoreload during development so latest (new) version local code library is reloaded on execution 
## can be commented out when local code development is complete to avoid any overhead
%reload_ext autoreload
%autoreload 2

## import libraries
import io
import time                          ## to enable delays between file saves
import numpy as np                   ## array manipulation
import matplotlib.pyplot as plt      ## for graphing
## import tensorflow
import tensorflow as tf              ## tensor, model functions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.losses import BinaryCrossentropy, SparseCategoricalCrossentropy
## local code library, - developed for these specific tasks and includes all references to MedMNIST specific library
import AMLS_common as ac

## Set base parameters
Including hyperparameter lists and data set specifics

In [2]:
## set up lists and parameters
hyper_list    = []
run_list      = []
batch_size    = 128                 ## batch size for datasets
patience      = 3                   ## number of overfitting epochs before terminating
threshold     = 0.1                 ## overfitting threshold
## control and environment (e.g. verbose) parameters
filebase      = "metrics/"          ## folder under current directory to store output files
verbose       = 0                   ## to control whether additional in process information is printed

## use these lists of values to grid test hyper parameter sensitivity                
epochs_list   = [15,20]                             ## set of epochs to run for
filter_list   = [64,128]                                ## main filter sizes to use
ks_list       = [3]                                 ## kernel size
lr_list       = [0.01]                        ## learning rates
ly_list       = [4,5]                               ## number of covolution layers
dr_list       = [0.25]                          ## selected dropout rates
st_list       = [2]                               ## stride list
loss_list     = ['sparse_categorical_crossentropy'] ## loss functions to use
optimise_list = ['Adam']                            ## optimisation functions
padding       = "same"
## now set up the required hyperparameter sets
for lr in lr_list:
    for ks in ks_list:
        for ep in epochs_list:
            for fi in filter_list:
                for ly in ly_list:
                    for dr in dr_list:
                        for st in st_list:
                            for op in optimise_list:
                                for ls in loss_list:
                                    parameter_set = ac.HyperParameters(learning_rate=lr, 
                                                                       kernel_size=ks, 
                                                                       num_epochs=ep, 
                                                                       num_filter=fi,
                                                                       layers=ly,
                                                                       dropout_rate=dr,
                                                                       strides=st,
                                                                       padding=padding,
                                                                       optimise=op,
                                                                       loss=ls)          
                                    hyper_list.append([parameter_set])        
## reshape parameters into a test grid that can be read using for loop
hyper_grid = [hp for sublist in hyper_list for hp in sublist]
print("Hyperparameter set combinations:",len(hyper_grid))

Hyperparameter set combinations: 8


## Load and preprocess the BloodMNIST Data
We load the dataset using the specifically developed common AMLS library. Uses default batch size.

In [3]:
## Loading the data file using common MedMINST loader
data_flag  = 'bloodmnist'                            ## defines which dataset to load
result_set = ac.medMNIST_load(data_flag,batch_size)  ## batch size currently hardwired

## check that the loader returned data correctly and then split out
if result_set != []:
    train_dataset = result_set[0]               ## training set
    test_dataset  = result_set[1]               ## test set
    val_dataset   = result_set[2]               ## validation set

if verbose == 1:
    print("\nSummary metrics for train_dataset")
    print("type:",type(val_dataset))
    print("length:",len(val_dataset))
    print("shape:",val_dataset)

Using downloaded and verified file: C:\Users\johnc\.medmnist\bloodmnist.npz
Using downloaded and verified file: C:\Users\johnc\.medmnist\bloodmnist.npz
Using downloaded and verified file: C:\Users\johnc\.medmnist\bloodmnist.npz


## Fit the model

Using each Hyperparameter in turn from a superset

In [4]:
## set up the variables to keep track of the hyperparameter combinations
iterations      = len(hyper_grid)                                     ## number of hyperparameter sets
countie         = 0                                                   ## interim count for iterations in loop
stop_overfit_cb = ac.StopOverfittingCallback(patience, threshold)     ## initialise overfitting callback
## Create instances of the dataclass from the list
for item in hyper_grid:
    countie += 1
    ## Define the model which is then run for all hyperparameters in set
    print("Run",countie,"/",iterations,"with",item)
    ## initialise tqdm callback
    tqdm_callback = ac.TqdmEpochProgress(total_epochs=item.num_epochs)
    
    ## Simple CNN model to support hyperparameter selection
    ## added desired number of layers
    if item.layers == 3:
            model = Sequential([
                Conv2D(item.num_filter*4, kernel_size=item.kernel_size,\
                       padding=item.padding,activation=item.default_activation,\
                       input_shape=(28, 28, 3)),                                             ## Input layer with larger num filter
                Conv2D(item.num_filter, kernel_size=item.kernel_size,\
                       padding=item.padding,activation=item.default_activation),             ## Second part of convolution layer 
                MaxPooling2D((2, 2),strides=item.strides),                                   ## Combined with pooling
                Dropout(item.dropout_rate),                                                  ## Initial dropout to reduce overfitting
                Conv2D(item.num_filter, kernel_size=item.kernel_size,\
                       padding=item.padding,activation=item.default_activation),             ## Another convolution layer 
                MaxPooling2D((2, 2),strides=item.strides),                                   ## Combined with pooling
                Flatten(),                                                                   ## Flatten
                Dense(item.num_filter*8,activation=item.default_activation),
                Dropout(item.dropout_rate*2),                                                ## Added larger dropout to reduce overfitting
                Dense(8, activation='softmax')                                               ## Output layer for 8 types 
            ])
        
    if item.layers == 4:
            model = Sequential([
                Conv2D(item.num_filter*4, kernel_size=item.kernel_size,\
                       padding=item.padding,activation=item.default_activation,\
                       input_shape=(28, 28, 3)),                                             ## Input layer with larger num filter
                Conv2D(item.num_filter, kernel_size=item.kernel_size,\
                       padding=item.padding,activation=item.default_activation),             ## Second part of convolution layer 
                MaxPooling2D((2, 2),strides=item.strides),                                   ## Combined with pooling
                Dropout(item.dropout_rate),                                                  ## Initial dropout to reduce overfitting
                Conv2D(item.num_filter, kernel_size=item.kernel_size,\
                       padding=item.padding,activation=item.default_activation),             ## Another convolution layer 
                Conv2D(item.num_filter, kernel_size=item.kernel_size,\
                       padding=item.padding,activation=item.default_activation),             ## With added convolution layer 
                MaxPooling2D((2, 2),strides=item.strides),                                   ## Combined with pooling
                Flatten(),                                                                   ## Flatten
                Dense(item.num_filter*8,activation=item.default_activation),
                Dropout(item.dropout_rate*2),                                                ## Added larger dropout to reduce overfitting
                Dense(8, activation='softmax')                                               ## Output layer for 8 types 
            ])
    
    if item.layers == 5:
            model = Sequential([
                Conv2D(item.num_filter*4, kernel_size=item.kernel_size,\
                       padding=item.padding,activation=item.default_activation,\
                       input_shape=(28, 28, 3)),                                             ## Input layer with larger num filter
                Conv2D(item.num_filter, kernel_size=item.kernel_size,\
                       padding=item.padding,activation=item.default_activation),             ## Second part of convolution layer 
                MaxPooling2D((2, 2),strides=item.strides),                                   ## Combined with pooling
                Dropout(item.dropout_rate),                                                  ## Initial dropout to reduce overfitting
                Conv2D(item.num_filter, kernel_size=item.kernel_size,\
                       padding=item.padding,activation=item.default_activation),             ## Another convolution layer 
                Conv2D(item.num_filter, kernel_size=item.kernel_size,\
                       padding=item.padding,activation=item.default_activation),             ## With added convolution layer 
                MaxPooling2D((2, 2),strides=item.strides),                                   ## Again reduce the features
                Conv2D(item.num_filter, kernel_size=item.kernel_size,\
                       padding=item.padding,activation=item.default_activation),             ## Another convolution layer 
                Conv2D(item.num_filter, kernel_size=item.kernel_size,\
                       padding=item.padding,activation=item.default_activation),             ## With added convolution layer 
                MaxPooling2D((2, 2),strides=item.strides),                                   ## Combined with pooling
                Flatten(),                                                                   ## Flatten
                Dense(item.num_filter*8,activation=item.default_activation),
                Dropout(item.dropout_rate*2),                                                ## Added larger dropout to reduce overfitting
                Dense(8, activation='softmax')                                               ## Output layer for 8 types 
            ])

    if verbose == 1:
        print(model.summary())
        
    ## Redirect the summary output to a string
    summary_string  = io.StringIO()
    model.summary(print_fn=lambda x: summary_string.write(x + "\n"))
    summary_content = summary_string.getvalue()
    summary_string.close()

    ## Compile the model
    model.compile(optimizer=item.optimise,                                                   
                  loss=item.loss,
                  metrics='acc')

    ## Fit the model
    history = model.fit(train_dataset,
                        validation_data=val_dataset, 
                        epochs=item.num_epochs, 
                        batch_size=batch_size, 
                        verbose=0,
                        callbacks = [tqdm_callback,stop_overfit_cb])
    
    ## Save results to files
    run_list.append(ac.hyper_process(history,summary_content,item))

print("Hyperparameter test run complete")

Run 1 / 8 with HyperParameters(learning_rate=0.01, kernel_size=3, num_epochs=15, optimise='Adam', loss='sparse_categorical_crossentropy', num_filter=64, strides=2, padding='same', dropout_rate=0.25, layers=4, default_activation='relu')


Epoch Progress: 100%|██████████| 15/15 [29:30<00:00, 118.02s/epoch, loss=0.243, acc=0.91, val_loss=0.233, val_acc=0.919] 


Run 2 / 8 with HyperParameters(learning_rate=0.01, kernel_size=3, num_epochs=15, optimise='Adam', loss='sparse_categorical_crossentropy', num_filter=64, strides=2, padding='same', dropout_rate=0.25, layers=5, default_activation='relu')


Epoch Progress:  13%|█▎        | 2/15 [03:54<25:28, 117.57s/epoch, loss=0.852, acc=0.675, val_loss=0.999, val_acc=0.627]

Overfitting detected at epoch 2: Loss Gap = 0.1463


...

Run 3 / 8 with HyperParameters(learning_rate=0.01, kernel_size=3, num_epochs=15, optimise='Adam', loss='sparse_categorical_crossentropy', num_filter=128, strides=2, padding='same', dropout_rate=0.25, layers=4, default_activation='relu')


Epoch Progress: 100%|██████████| 15/15 [1:29:38<00:00, 358.54s/epoch, loss=0.188, acc=0.932, val_loss=0.26, val_acc=0.921] 


Run 4 / 8 with HyperParameters(learning_rate=0.01, kernel_size=3, num_epochs=15, optimise='Adam', loss='sparse_categorical_crossentropy', num_filter=128, strides=2, padding='same', dropout_rate=0.25, layers=5, default_activation='relu')


Epoch Progress:   7%|▋         | 1/15 [14:41<3:25:40, 881.49s/epoch, loss=1.54, acc=0.403, val_loss=0.947, val_acc=0.633]

KeyboardInterrupt: 

In [ ]:
## Get best hyperparameter sets and both print them out and save them to parameter files that can be fed to Tune model runs
run_df,best_run,best_run2,best_run3 = ac.analyse_run(run_list," ",filebase)
print("\nRun satisfying both smallest min_loss and largest max_acc:")
if len(best_run) > 0:
    ac.process_best_run(best_run)
print("\n")
print("\nRun with largest max_acc that is plateau or increasing:")
if len(best_run2) > 0:
    ac.process_best_run(best_run2)
print("\n")
print("\nRun with smallest min_loss that is plateau or decreasing:")
if len(best_run3) > 0:
    ac.process_best_run(best_run3)

if len(run_df)>1:
    feature_importance,coef = ac.analyse_hyperparameters(run_df)
    print("\nImpact of Hyperparameters on Accuracy (from Linear Regression):")
    print(coef)
    print("\nHyperparameter Importance for Accuracy (from Random Forest):")
    print(feature_importance)
else:
    print("\n")
    print('Suppressed feature analysis as train set too small')